In [17]:
import os
import ebooklib
from ebooklib import epub
import glob
from bs4 import BeautifulSoup

In [6]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters


def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output


def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text = chap2text(html)
        Output.append(text)
    return Output


def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [31]:
files = glob.glob('texts/**/*.epub', recursive=True)
to_overwrite = None
for file in files:
    text = epub2text(file)
    title = file.split("\\")[-1].split(".")[0]
    author = file.split("\\")[-2]
    txt_dir = os.path.join('texts', 'txt', author) 
    if not os.path.exists(txt_dir):
        os.makedirs(txt_dir)
    txt_path = os.path.join(txt_dir, title + '.txt')
    if os.path.exists(txt_path):
        if to_overwrite is None:
            print("Перезаписывать файлы? (y/n)")
            to_overwrite = input()
        if to_overwrite != 'y':
            continue
    with open(txt_path, 'w', encoding='utf-8') as new_txt_file:
        for part in text:
            new_txt_file.write(part)
            print(txt_path)

Перезаписывать файлы? (y/n)
n
